# Traces for draft-lake-authz

Requirements:

```python
pip install cryptography==3.4.7 cbor2==5.3.0 rich==10.6.0 hkdf==0.0.3
```

In [17]:
import rich, cbor2, hkdf, hashlib
from cryptography.hazmat.primitives import asymmetric, serialization
from cryptography.hazmat.primitives.ciphers import aead
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from binascii import hexlify, unhexlify

### Common functions, keys, and creds

In [23]:
def format_tv(tv, fmt, nokeys=False, prefix=""):
    for k, v in tv.items():
        if k[0] == "_" or (nokeys and k in ["static_keys", "ephemeral_keys"]):
            continue
        elif type(v) == dict:
            print(f"\n# {k}")
            format_tv(v, fmt, nokeys, prefix)
        elif type(v) == int:
            if fmt == "rust":
                print(f'const {prefix}{k.upper()}_TV: u8 = {v};')
            elif fmt == "rust":
                print(f'static const uint8_t {prefix}{k.upper()} = {v};')
            elif fmt == "python":
                print(f'{prefix}{k:<8} = {v}')
        else:
            if fmt == "rust":
                print(f'const {prefix}{k.upper()}_TV: &[u8] = &hex!("{v}");')
            elif fmt == "c":
                pairs = ["0x"+"".join(v[e:e+2])+", " for e in range(0, len(v), 2)]
                c_values = "".join(pairs)[:-2]
                print(f'static const uint8_t {prefix}{k.upper()}[] = {{{c_values}}};')
                print(f'static const size_t {prefix}{k.upper()}_LEN = {len(pairs)};')
            elif fmt == "python":
                print(f'{prefix}{k:<8} = "{v}"')

def add_new_keys(tv):
    def as_hex(k):
        return hex(k)[2:]
    def new_keypair_dx_testvector(entity_name):
        private_key = ec.generate_private_key(ec.SECP256R1(), backend=default_backend())
        x = private_key.public_key().public_numbers().x
        y = private_key.public_key().public_numbers().y
        d = private_key.private_numbers().private_value
        return {f"{entity_name}": as_hex(d), f"G_{entity_name}": as_hex(x), f"G_{entity_name}_y": as_hex(y)}

    tv["static_keys"] = {}
    tv["ephemeral_keys"] = {}
    for a in ["U", "V", "W"]:
        tv["static_keys"].update(new_keypair_dx_testvector(a))
    for a in ["X", "Y", "Z"]:
        tv["ephemeral_keys"].update(new_keypair_dx_testvector(a))

    return tv

def add_creds(tv):
    cred_v = cbor2.dumps({
        2: "example.edu",
        8: {
            1: {
                1: 2,
                2: b'\x32',
                -1: 1,
                -2: unhexlify(tv["static_keys"]["G_V"]),
                -3: unhexlify(tv["static_keys"]["G_V_y"]),
            }
        }
    }).hex() # lake-traces-07
    tv.update({
        "creds": {
            "CRED_V": cred_v,
        }
    })
    return tv

keys_tv = {
    'static_keys': {
        'U': 'fb13adeb6518cee5f88417660841142e830a81fe334380a953406a1305e8706b', # lake-traces-07
        'G_U': 'ac75e9ece3e50bfc8ed60399889522405c47bf16df96660a41298cb4307f7eb6',
        'G_U_y': '6e5de611388a4b8a8211334ac7d37ecb52a387d257e6db3c2a93df21ff3affc8',
        'V': '72cc4761dbd4c78f758931aa589d348d1ef874a7e303ede2f140dcf3e6aa4aac', # lake-traces-07
        'G_V': 'bbc34960526ea4d32e940cad2a234148ddc21791a12afbcbac93622046dd44f0',
        'G_V_y': '4519e257236b2a0ce2023f0931f1f386ca7afda64fcde0108c224c51eabf6072',
        'W': '4E5E15AB35008C15B89E91F9F329164D4AACD53D9923672CE0019F9ACD98573F',
        'G_W': 'FFA4F102134029B3B156890B88C9D9619501196574174DCB68A07DB0588E4D41',
        'G_W_y': 'BD08125C1A5E9C4F4AA60198A9F897EB656784DE50C0FE840FE3683FC20C295C'
    },
    'ephemeral_keys': {
        'X': '368ec1f69aeb659ba37d5a8d45b21bdc0299dceaa8ef235f3ca42ce3530f9525', # lake-traces-07
        'G_X': '8af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6',
        'G_X_y': '51e8af6c6edb781601ad1d9c5fa8bf7aa15716c7c06a5d038503c614ff80c9b3',
        'Y': 'e2f4126777205e853b437d6eaca1e1f753cdcc3e2c69fa884b0a1a640977e418', # lake-traces-07
        'G_Y': '419701d7f00a26c2dc587a36dd752549f33763c893422c8ea0f955a13a4ff5d5',
        'G_Y_y': '5e4f0dd8a3da0baa16b9d3ad56a0c1860a940af85914915e25019b402417e99d',
        'Z': '644658D815CBCA8EA863090A2D498990B5C75357A729231EC3DE7DF5A7AFE49E',
        'G_Z': '6B67C90638924C4AE8472CA6FB9A90BE5F43132753346379C672972D323F7A41',
        'G_Z_y': 'FA1EFAD24A287B1FEF04683B5B24963A107067541B2E4766088552EE11337D87'
    },
}
# keys_tv = add_new_keys(keys_tv) # uncomment to generate a new set of keys

keys_tv = add_creds(keys_tv)

format_tv(keys_tv, "rust")


# static_keys
const U_TV: &[u8] = &hex!("fb13adeb6518cee5f88417660841142e830a81fe334380a953406a1305e8706b");
const G_U_TV: &[u8] = &hex!("ac75e9ece3e50bfc8ed60399889522405c47bf16df96660a41298cb4307f7eb6");
const G_U_Y_TV: &[u8] = &hex!("6e5de611388a4b8a8211334ac7d37ecb52a387d257e6db3c2a93df21ff3affc8");
const V_TV: &[u8] = &hex!("72cc4761dbd4c78f758931aa589d348d1ef874a7e303ede2f140dcf3e6aa4aac");
const G_V_TV: &[u8] = &hex!("bbc34960526ea4d32e940cad2a234148ddc21791a12afbcbac93622046dd44f0");
const G_V_Y_TV: &[u8] = &hex!("4519e257236b2a0ce2023f0931f1f386ca7afda64fcde0108c224c51eabf6072");
const W_TV: &[u8] = &hex!("4E5E15AB35008C15B89E91F9F329164D4AACD53D9923672CE0019F9ACD98573F");
const G_W_TV: &[u8] = &hex!("FFA4F102134029B3B156890B88C9D9619501196574174DCB68A07DB0588E4D41");
const G_W_Y_TV: &[u8] = &hex!("BD08125C1A5E9C4F4AA60198A9F897EB656784DE50C0FE840FE3683FC20C295C");

# ephemeral_keys
const X_TV: &[u8] = &hex!("368ec1f69aeb659ba37d5a8d45b21bdc0299dceaa8ef235f3ca42ce3530f9525");

### Crypto functions

In [6]:
def p256_ecdh(d_hex, x_hex, y_hex):
    private_key = ec.derive_private_key(int(d_hex, 16), ec.SECP256R1(), default_backend())
    # NOTE: rust uses the compressed form of the public key (without the y coordinate), but the result should be the same
    public_key = ec.EllipticCurvePublicNumbers(
        int(x_hex, 16),
        int(y_hex, 16),
        ec.SECP256R1()
    ).public_key(default_backend())
    return private_key.exchange(ec.ECDH(), public_key).hex()

def hkdf_extract(salt, ikm):
    return hkdf.hkdf_extract(unhexlify(salt), unhexlify(ikm), hash=hashlib.sha256).hex()

def hkdf_expand(prk, info, length):
    return hkdf.hkdf_expand(unhexlify(prk), unhexlify(info), length, hash=hashlib.sha256).hex()

def aes_ccm_encrypt_tag_8(key, iv, enc_structure, plaintext):
    return aead.AESCCM(unhexlify(key), tag_length=8).encrypt(unhexlify(iv), unhexlify(plaintext), unhexlify(enc_structure)).hex()

def sha256_digest(message):
    return hashlib.sha256(unhexlify(message)).hexdigest()

import unittest
class Test(unittest.TestCase):
    def test_ecdh(self):
        self.assertEqual(
            p256_ecdh(keys_tv["ephemeral_keys"]["X"], keys_tv["static_keys"]["G_W"], keys_tv["static_keys"]["G_W_y"]), 
            p256_ecdh(keys_tv["static_keys"]["W"], keys_tv["ephemeral_keys"]["G_X"], keys_tv["ephemeral_keys"]["G_X_y"]), 
        )
unittest.main(argv=[''], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### EAD_1 traces

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-device-enrollment-server-u-

In [15]:

def add_enc_id(tv):
    salt = ""
    g_xw = p256_ecdh(tv["ephemeral_keys"]["X"], tv["static_keys"]["G_W"], tv["static_keys"]["G_W_y"])
    prk = hkdf_extract(salt, g_xw)
    k_1_info = (cbor2.dumps(0)+cbor2.dumps(b'')+cbor2.dumps(16)).hex() # info is (0, b'', 16) # FIXME[draft] make 'length' explicit
    iv_1_info = (cbor2.dumps(1)+cbor2.dumps(b'')+cbor2.dumps(13)).hex() # info is (1, b'', 13) # FIXME[draft] make 'length' explicit
    k_1 = hkdf_expand(prk, k_1_info, 16)
    iv_1 = hkdf_expand(prk, iv_1_info, 13)
    plaintext = cbor2.dumps(unhexlify(tv["input"]["ID_U"])).hex() # (ID_U: bstr)
    _ss = tv["input"]["SS"].to_bytes(1, byteorder='big')
    enc_structure = cbor2.dumps(["Encrypt0", b'', _ss]).hex()
    enc_id = aes_ccm_encrypt_tag_8(k_1, iv_1, enc_structure, plaintext)
    tv.update({
        "enc_id": {
            "enc_id": enc_id,
            "salt": salt,
            "g_xw": g_xw,
            "prk": prk,
            "k_1_info": k_1_info,
            "iv_1_info": iv_1_info,
            "k_1": k_1,
            "iv_1": iv_1,
            "plaintext": plaintext,
            "enc_structure": enc_structure,
        }
    })

    return tv

def add_voucher_info(tv):
    # (LOC_W: tstr, ENC_ID: bstr)
    voucher_info_seq = (cbor2.dumps(unhexlify(tv["input"]["LOC_W"]).decode()) + cbor2.dumps(unhexlify(tv["enc_id"]["enc_id"]))).hex()
    voucher_info = cbor2.dumps(unhexlify(voucher_info_seq)).hex()
    tv.update({
        "voucher_info": {
            "voucher_info": voucher_info,
            "voucher_info_seq": voucher_info_seq,
        }
    })
    return tv

def add_ead1(tv):
    label = "01"
    value = tv["voucher_info"]["voucher_info"]
    ead1 = label + value
    tv.update({
        "ead1": {
            "ead1": ead1,
            "ead1_label": label,
            "ead1_value": value,
        }
    })
    return tv

ead1_tv = {
    "input": {
        "LOC_W": "coap://enrollment.server".encode().hex(),
        "ID_U": cbor2.dumps({4: b'\x2B'}).hex(),
        "SS": 2,
    }
}
ead1_tv.update(keys_tv) # using existing keys
# ead1_tv = add_new_keys(ead1_tv) # uncomment to generate a new set of keys

ead1_tv = add_enc_id(ead1_tv)
ead1_tv = add_voucher_info(ead1_tv)
ead1_tv = add_ead1(ead1_tv)

format_tv(ead1_tv, "rust", nokeys=True)


# input
const LOC_W_TV: &[u8] = &hex!("636f61703a2f2f656e726f6c6c6d656e742e736572766572");
const ID_U_TV: &[u8] = &hex!("a104412b");
const SS_TV: u8 = 2;

# creds
const CRED_V_TV: &[u8] = &hex!("a2026b6578616d706c652e65647508a101a501020241322001215820bbc34960526ea4d32e940cad2a234148ddc21791a12afbcbac93622046dd44f02258204519e257236b2a0ce2023f0931f1f386ca7afda64fcde0108c224c51eabf6072");

# enc_id
const ENC_ID_TV: &[u8] = &hex!("da9784962883c96ed01ff122c3");
const SALT_TV: &[u8] = &hex!("");
const G_XW_TV: &[u8] = &hex!("03a658e9628c79c3f1e59239ca5e604953d11e01c2a442823c944da6682d0b6c");
const PRK_TV: &[u8] = &hex!("d40f1601b577dbe7827bb3a20e0d16f7231c3a25225c1ed733f9094050d59666");
const K_1_INFO_TV: &[u8] = &hex!("004010");
const IV_1_INFO_TV: &[u8] = &hex!("01400d");
const K_1_TV: &[u8] = &hex!("6f2a9112801a5011aa33576b5c7862ad");
const IV_1_TV: &[u8] = &hex!("d31bc0d128349f290e79f0bde3");
const PLAINTEXT_TV: &[u8] = &hex!("44a104412b");
const ENC_STRUCTURE_TV: &[u8] = &hex!("8368456

### Voucher_Request (VREQ) traces

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-authenticator-enrollment-se

In [16]:
def add_voucher_request(tv):
    voucher_request = cbor2.dumps([
            unhexlify(tv["input"]["MESSAGE_1_WITH_EAD"])
        ]).hex()
    tv.update({
        "voucher_request": {
            "voucher_request": voucher_request,
        }
    })
    return tv

voucher_request_tv = {
    "input": {
        "EAD_1_VALUE": ead1_tv["ead1"]["ead1_value"],
        "MESSAGE_1_WITH_EAD": "0382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b637" + ead1_tv["ead1"]["ead1"],
    }
}

voucher_request_tv = add_voucher_request(voucher_request_tv)

format_tv(voucher_request_tv, "rust")


# input
const EAD_1_VALUE_TV: &[u8] = &hex!("58287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const MESSAGE_1_WITH_EAD_TV: &[u8] = &hex!("0382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");

# voucher_request
const VOUCHER_REQUEST_TV: &[u8] = &hex!("8158520382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");


### Voucher_Response (VRES) traces

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-voucher

In [17]:
def add_voucher_response(tv):
    h_message_1 = sha256_digest(tv["input"]["MESSAGE_1_WITH_EAD"])
    voucher_input = (cbor2.dumps(unhexlify(h_message_1)) + cbor2.dumps(unhexlify(tv["input"]["CRED_V"]))).hex()
    label = 2
    context = cbor2.dumps(unhexlify(voucher_input)).hex()
    mac_length = 8
    info = (cbor2.dumps(label) + unhexlify(context) + cbor2.dumps(mac_length)).hex()
    voucher_mac = hkdf_expand(tv["input"]["PRK"], info, mac_length)
    voucher = cbor2.dumps(unhexlify(voucher_mac)).hex()
    voucher_response = cbor2.dumps([
        unhexlify(tv["input"]["MESSAGE_1_WITH_EAD"]),
        unhexlify(voucher),
    ]).hex()
    tv.update({
        "voucher_response": {
            "voucher_response": voucher_response,
            "h_message_1": h_message_1,
            "voucher_input": voucher_input,
            "label": label,
            "context": context,
            "mac_length": mac_length,
            "info": info,
            "voucher_mac": voucher_mac,
            "voucher": voucher,
        }
    })
    return tv

voucher_tv = {
    "input": {
        "VOUCHER_REQUEST": voucher_request_tv["voucher_request"]["voucher_request"],
        "MESSAGE_1_WITH_EAD": voucher_request_tv["input"]["MESSAGE_1_WITH_EAD"],
        "CRED_V": keys_tv["creds"]["CRED_V"],
        "PRK": ead1_tv["enc_id"]["prk"],
    }
}
voucher_tv = add_voucher_response(voucher_tv)

format_tv(voucher_tv, "rust")


# input
const VOUCHER_REQUEST_TV: &[u8] = &hex!("8158520382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const MESSAGE_1_WITH_EAD_TV: &[u8] = &hex!("0382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const CRED_V_TV: &[u8] = &hex!("a2026b6578616d706c652e65647508a101a501020241322001215820bbc34960526ea4d32e940cad2a234148ddc21791a12afbcbac93622046dd44f02258204519e257236b2a0ce2023f0931f1f386ca7afda64fcde0108c224c51eabf6072");
const PRK_TV: &[u8] = &hex!("d40f1601b577dbe7827bb3a20e0d16f7231c3a25225c1ed733f9094050d59666");

# voucher_response
const VOUCHER_RESPONSE_TV: &[u8] = &hex!("8258520382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c34948c7836

### EAD_2 traces

This one is rather unecessary, sinde EAD_2 = Voucher.

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-voucher

In [18]:
def add_ead2(tv):
    tv.update({
        "ead2": {
            "ead2_value": tv["input"]["VOUCHER"],
        }
    })
    return tv

ead2_tv = {
    "input": {
        "VOUCHER": voucher_tv["voucher_response"]["voucher"],
    }
}
ead2_tv = add_ead2(ead2_tv)

format_tv(ead2_tv, "rust")


# input
const VOUCHER_TV: &[u8] = &hex!("48c783671337f75bd5");

# ead2
const EAD2_VALUE_TV: &[u8] = &hex!("48c783671337f75bd5");


## Traces for stateless operation

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-stateless-operation-of-v

The variables are identified with a **SLO_** (stateless operation) prefix.

### SLO: Voucher_Request (VREQ) traces

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-authenticator-enrollment-se

In [19]:
def add_slo_voucher_request(tv):
    voucher_request = cbor2.dumps([
            unhexlify(tv["input"]["MESSAGE_1_WITH_EAD"]),
            unhexlify(tv["input"]["OPAQUE_STATE"]),
        ]).hex()
    tv.update({
        "voucher_request": {
            "voucher_request": voucher_request,
        }
    })
    return tv

slo_voucher_request_tv = {
    "input": {
        "OPAQUE_STATE": cbor2.dumps(["fe80::b834:d60b:796f:8de0", 35821]).hex(), # [ORIGIN_IPADDR, PORT]
    }
}
slo_voucher_request_tv["input"].update(voucher_request_tv["input"])

slo_voucher_request_tv = add_slo_voucher_request(slo_voucher_request_tv)

format_tv(slo_voucher_request_tv, "rust", prefix="SLO_")


# input
const SLO_OPAQUE_STATE_TV: &[u8] = &hex!("827819666538303a3a623833343a643630623a373936663a38646530198bed");
const SLO_EAD_1_VALUE_TV: &[u8] = &hex!("58287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const SLO_MESSAGE_1_WITH_EAD_TV: &[u8] = &hex!("0382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");

# voucher_request
const SLO_VOUCHER_REQUEST_TV: &[u8] = &hex!("8258520382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3581f827819666538303a3a623833343a643630623a373936663a38646530198bed");


### SLO: Voucher_Response (VRES) traces

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-voucher

In [20]:
def add_slo_voucher_response(tv):
    voucher_response = cbor2.dumps([
        unhexlify(tv["input"]["MESSAGE_1_WITH_EAD"]),
        unhexlify(tv["input"]["VOUCHER"]),
        unhexlify(tv["input"]["OPAQUE_STATE"]),
    ]).hex()
    tv.update({
        "voucher_response": {
            "voucher_response": voucher_response,
        }
    })
    return tv

slo_voucher_tv = {
    "input": {
        "OPAQUE_STATE": slo_voucher_request_tv["input"]["OPAQUE_STATE"],
    }
}

# copy fields over from voucher_tv (non-slo)
slo_voucher_tv["input"].update(voucher_tv["input"])
slo_voucher_tv["input"]["VOUCHER"] = voucher_tv["voucher_response"]["voucher"]

slo_voucher_tv = add_slo_voucher_response(slo_voucher_tv)

format_tv(slo_voucher_tv, "rust", prefix="SLO_")


# input
const SLO_OPAQUE_STATE_TV: &[u8] = &hex!("827819666538303a3a623833343a643630623a373936663a38646530198bed");
const SLO_VOUCHER_REQUEST_TV: &[u8] = &hex!("8158520382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const SLO_MESSAGE_1_WITH_EAD_TV: &[u8] = &hex!("0382060258208af6f430ebe18d34184017a9a11bf511c8dff8f834730b96c1b7c8dbca2fc3b6370158287818636f61703a2f2f656e726f6c6c6d656e742e7365727665724dda9784962883c96ed01ff122c3");
const SLO_CRED_V_TV: &[u8] = &hex!("a2026b6578616d706c652e65647508a101a501020241322001215820bbc34960526ea4d32e940cad2a234148ddc21791a12afbcbac93622046dd44f02258204519e257236b2a0ce2023f0931f1f386ca7afda64fcde0108c224c51eabf6072");
const SLO_PRK_TV: &[u8] = &hex!("d40f1601b577dbe7827bb3a20e0d16f7231c3a25225c1ed733f9094050d59666");
const SLO_VOUCHER_TV: &[u8] = &hex!("48c783671337f75bd5");

# voucher_response
const SLO_VOUCHER_RESPONSE_TV: &[u8] =